In [1]:
import numpy as np
import pandas as pd
import torch
# ! pip install sentence_transformers
# from sentence_transformers import SentenceTransformer
from sklearn.model_selection import train_test_split
import gensim
from gensim.test.utils import common_texts
from gensim.models import Word2Vec

# Load the data

In [2]:
data = pd.read_csv("drive/MyDrive/Colab Notebooks/data_processed.csv")
data

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,as a woman you shouldnt complain about cleani...
1,1,3,0,3,0,1,boy dats coldtyga dwn bad for cufin dat hoe i...
2,2,3,0,3,0,1,dawg you ever fuck a bitch and she start to c...
3,3,3,0,2,1,1,she lok like a trany
4,4,6,0,6,0,1,the shit you hear about me might be true or i...
...,...,...,...,...,...,...,...
24768,25291,3,0,2,1,1,yous a muthafin lie right his tl is trash now ...
24769,25292,3,0,1,2,2,youve gone and broke the wrong heart baby and ...
24770,25294,3,0,3,0,1,young buck wana eat dat niguh like i aint fuck...
24771,25295,6,0,6,0,1,you got wild bitches telin you lies


In [3]:
# model = SentenceTransformer('all-MiniLM-L6-v2')
sentences = data["tweet"]
# sentence_embeddings = model.encode(sentences)
words = [list(s.lower().split()) for s in sentences]

In [4]:
# w2v = gensim.models.KeyedVectors.load_word2vec_format('drive/MyDrive/Colab Notebooks/GoogleNews-vectors-negative300.bin', binary=True)  
w2v = Word2Vec(sentences=words, size=384, window=5, min_count=1, workers=4)
# import gensim.downloader
# w2v = gensim.downloader.load('glove-twitter-25')
# w2v.train(words,total_examples=len(words),epochs=15)
sentence_embeddings = [torch.as_tensor([w2v.wv[word] for word in list(s.lower().split())]) for s in sentences]

<ipython-input-4-d76c2a48e765>:6: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  sentence_embeddings = [torch.as_tensor([w2v.wv[word] for word in list(s.lower().split())]) for s in sentences]


In [5]:
label = data["class"]
assert len(label) == len(sentence_embeddings)

# Train-validation-test split

In [6]:
X_train, X_rem, y_train, y_rem = train_test_split(sentence_embeddings,label, train_size=0.7,stratify=label)
X_valid, X_test, y_valid, y_test = train_test_split(X_rem,y_rem, test_size=0.5)

In [7]:
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence

class Tweet(Dataset):
    def __init__(self, X,y, transform=None):
        self.X = pad_sequence(X,batch_first=True)
        self.y = torch.as_tensor(np.array(y))
        self.transform = transform

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        sentence = self.X[idx]
        label = self.y[idx]
        sample = {'sentence': sentence, 'label': label}

        if self.transform:
            sample = self.transform(sample)

        return sample

# Define the model

In [8]:
hidden_size = 128
output_size = 3
batch_size = 50

valid_loss = []

In [9]:
import torch.nn as nn
import torch
from torch.autograd import Variable


class lstm(nn.Module):
    def __init__(self, embedding_length=384):
        super(lstm, self).__init__()
  
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.batch_size = batch_size
        self.lstm = nn.LSTM(embedding_length, self.hidden_size,batch_first=True)
        self.label = nn.Linear(self.hidden_size, self.output_size)
        self.to("cuda")


    def forward(self,input):

        input = input.to("cuda")


        h_0 = Variable(torch.zeros(1, len(input), self.hidden_size).cuda())
        c_0 = Variable(torch.zeros(1, len(input), self.hidden_size).cuda())


        output, (final_hidden_state, final_cell_state) = self.lstm(input, (h_0, c_0))

        return self.label(final_hidden_state[-1])
    

    def fit(self, train_loader, epochs=100,lr=1e-5,interval=100):
        criterion = torch.nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(self.parameters(), lr=lr)

        for epoch in range(epochs):
            running_loss = 0.0
            self.train()

            for i_batch, sample in enumerate(train_loader):
                sentences, labels = sample['sentence'].to("cuda"), sample['label'].to("cuda")

                outputs = self(sentences)
                loss = criterion(
                    outputs,
                    labels,
                )

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                running_loss += loss.item()

                if i_batch % interval == 0 and i_batch > 0:
                    print(f"epoch:{epoch},batch: {i_batch}, loss: {running_loss/interval}")
                    running_loss = 0
            valid_loss.append([epoch,self.validate(valid_loader)])

    def validate(self, valid_loader):
        running_loss = 0
        val_count = 0
        self.eval()
        criterion = torch.nn.CrossEntropyLoss()

        for i_batch, sample in enumerate(valid_loader):
            sentences, labels = sample['sentence'].to("cuda"), sample['label'].to("cuda")

            outputs = self(sentences)
            loss = criterion(
                outputs,
                labels,
            )

            running_loss += loss.item()
            val_count += 1

        avg_loss = running_loss / val_count
        return avg_loss

    def accuracy(self,test_loader):
        self.eval()
        for i_batch, sample in enumerate(test_loader):
            sentences, labels = sample['sentence'], sample['label']
    
            outputs = self(sentences)
            pred = np.array([np.argmax(outputs[i].cpu().detach().numpy()) for i in range(len(outputs))])
            assert len(pred) == len(labels)

            return np.sum(labels.numpy() == pred) / len(pred)


    def predict(self,sentence):
        vec = torch.as_tensor([w2v.wv[word] for word in list(sentence.lower().split())])
        vec = vec.unsqueeze(0)
        index =  np.argmax(self(vec).cpu().detach().numpy()) 

        label_to_senti = {0:"hate speech",1:"offensive language",2:"neither"}
        return label_to_senti[index]

In [10]:
train_data = Tweet(X_train, y_train)
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=0,drop_last=True)

valid_data = Tweet(X_valid, y_valid)
valid_loader = DataLoader(valid_data, batch_size=batch_size, shuffle=True, num_workers=0,drop_last=True)

test_data = Tweet(X_test, y_test)
test_loader = DataLoader(test_data, batch_size=len(X_test), shuffle=True, num_workers=0,drop_last=True)

In [11]:
lstm_model = lstm()
valid_loss = []
lstm_model.fit(train_loader=train_loader)

In [12]:
import matplotlib.pyplot as plt
plt.plot(np.array(valid_loss)[:,0],np.array(valid_loss)[:,1])
plt.xlabel("epoch")
plt.ylabel("valid loss")

IndexError: ignored

In [13]:
lstm_model.accuracy(test_loader)

0.16630785791173305

In [14]:
lstm_model.predict("you are a dog")

'neither'